In [285]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import cPickle as pickle
import bottleneck as bn

In [2]:
%%time
train_df = pd.read_csv('train.csv')
artist_df = pd.read_csv('artists.csv')
profile_df = pd.read_csv('profiles.csv')

Wall time: 16.9 s


In [3]:
profile_df.set_index('user', inplace=True)

In [4]:
print profile_df.shape
profile_df.head()

(233286, 3)


,sex,age,country
user,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden
5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States
63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany
02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands


In [5]:
%%time
for k in range(10):
    profile_df[str(10*k) + 's'] = ((profile_df.age >= 10*k) & (profile_df.age < 10*k+10)).astype(int)
print profile_df.shape

(233286, 13)
Wall time: 197 ms


In [6]:
%%time
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.sex)], axis=1)
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.age)], axis=1)
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.country)], axis=1)

Wall time: 3.3 s


In [7]:
del profile_df['sex']
del profile_df['age']
del profile_df['country']

In [8]:
print profile_df.shape
profile_df.head()

(233286, 364)


,0s,10s,20s,30s,40s,50s,60s,70s,80s,90s,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5909125332c108365a26ccf0ee62636eee08215c,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63268cce0d68127729890c1691f62d5be5abd87c,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02871cd952d607ba69b64e2e107773012c708113,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
m = sparse.lil_matrix(profile_df.values)
m

<233286x364 sparse matrix of type '<type 'numpy.float64'>'
	with 823169 stored elements in LInked List format>

In [15]:
cols = list(profile_df.columns)
artist_list = [tup[1] for tup in artist_df.itertuples()]

cols.extend(artist_list)
m = sparse.hstack((m, sparse.csr_matrix((m.shape[0], len(artist_list)))), format='lil')
m

<233286x2364 sparse matrix of type '<type 'numpy.float64'>'
	with 823169 stored elements in LInked List format>

In [16]:
col_dict = {col:i for i,col in enumerate(cols)}

In [17]:
row_dict = {row:i for i, row in enumerate(profile_df.index)}

In [29]:
%%time
for row in train_df.itertuples():
    m[row_dict[row[1]], col_dict[row[2]]] = row[3]

Wall time: 1min 5s


In [32]:
m = m.tocsr()

In [36]:
%%time
with open('data.p', 'w') as p:
    pickle.dump((row_dict, col_dict, m), p)

Wall time: 24.7 s


###Clustering

In [163]:
from sklearn.decomposition import PCA, SparsePCA, MiniBatchSparsePCA, NMF
from sklearn.preprocessing import MaxAbsScaler

In [104]:
scaler = MaxAbsScaler()
m = scaler.fit_transform(m)

In [105]:
pca = MiniBatchSparsePCA(n_components=10)

In [112]:
%%time
pca.fit(m[0:100000,:].toarray())

Wall time: 21.3 s


MiniBatchSparsePCA(alpha=1, batch_size=3, callback=None, method='lars',
          n_components=10, n_iter=100, n_jobs=1, random_state=None,
          ridge_alpha=0.01, shuffle=True, verbose=False)

In [150]:
print map(lambda k : sum(pca.components_[k] != 0), range(10))
print np.array(cols)[pca.components_[9]!=0]

[0, 1, 0, 0, 2, 0, 0, 0, 1, 21]
['20s' 'f' 'm' '20.0' 'Australia' 'Brazil' 'Canada' 'Finland' 'France'
 'Germany' 'Italy' 'Mexico' 'Netherlands' 'Norway' 'Poland'
 'Russian Federation' 'Spain' 'Sweden' 'Turkey' 'United Kingdom'
 'United States']


In [280]:
nmf = NMF(n_components=10, init='nndsvd', l1_ratio=1)

In [282]:
%%time
nmf.fit(m)

Wall time: 11 s


NMF(alpha=0.0, beta=1, eta=0.1, init='nndsvd', l1_ratio=1, max_iter=200,
  n_components=10, nls_max_iter=2000, random_state=None, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [283]:
np.max(nmf.components_[:,364:])

0.042262344213656107

In [284]:
check_max = nmf.components_ == np.max(nmf.components_[:,364:])
dim0 = [tup for tup in enumerate(check_max.sum(axis=1)) if tup[1]==1][0][0]
dim1 = [tup for tup in enumerate(check_max.sum(axis=0)) if tup[1]==1][0][0]
assert(nmf.components_[dim0,dim1] == np.max(nmf.components_[:,364:]))

In [302]:
best_indices = map(lambda v : list(bn.argpartsort(-v,20)[0:20]), nmf.components_[:,364:])

In [307]:
artist_df.set_index('artist').loc[np.array(cols[364:])[np.array(best_indices[9])],:]

,name
artist,
0af78501-5647-4c18-9a0d-66ac8789e13b,Beirut
a47c3aa2-7d87-475c-a2c7-1e2047dafb09,Gnarls Barkley
14b22b4b-06d5-4b82-8284-29d29b58945f,Air
10adbe5e-a2c0-4bf3-8249-2b4cbf6e6ca8,Massive Attack
52074ba6-e495-4ef3-9bb4-0703888a9f68,Arcade Fire
6ffb8ea9-2370-44d8-b678-e9237bbd347b,Kings of Leon
bf710b71-48e5-4e15-9bd6-96debb2e4e98,The Knife
f1b525b4-ddd0-4d39-85b2-d8fa26a7f279,Feist
cf0f4547-ffbd-4011-98ad-0bec9ba022db,Kings of Convenience


In [299]:
[np.array(cols[364:])[np.array(row)] for row in best_indices]

[array(['65f4f0c5-ef9e-490c-aee3-909e7ae6b2ab',
        '14b22b4b-06d5-4b82-8284-29d29b58945f',
        'a47c3aa2-7d87-475c-a2c7-1e2047dafb09',
        '618b6900-0618-4f1e-b835-bccb17f84294',
        '83d91898-7763-47d7-b03b-b92132375c47',
        '35723b60-732e-4bd8-957f-320b416e7b7f',
        'a9044915-8be3-4c7e-b11f-9e2d2ea0a91e',
        'cc0b7089-c08d-4c10-b6b0-873582c17fd6',
        '10adbe5e-a2c0-4bf3-8249-2b4cbf6e6ca8',
        '6ffb8ea9-2370-44d8-b678-e9237bbd347b'], 
       dtype='|S36'), array(['0af78501-5647-4c18-9a0d-66ac8789e13b',
        '6e0c7c0e-cba5-4c2c-a652-38f71ef5785d',
        '6ffb8ea9-2370-44d8-b678-e9237bbd347b',
        'f82f3a3e-29c2-42ca-b589-bc5dc210fa9e',
        'f1b525b4-ddd0-4d39-85b2-d8fa26a7f279',
        'c485632c-b784-4ee9-8ea1-c5fb365681fc',
        'e795e03d-b5d5-4a5f-834d-162cfb308a2c',
        '2119beb8-6ac5-4f21-82a4-b831c90c0024',
        '05755bf1-380c-487f-983f-d1a02401fa28',
        'cf0f4547-ffbd-4011-98ad-0bec9ba022db'], 
       dtype='|

In [ ]:
best_indices = map(lambda v : list(bn.argpartsort(-v,num_best)[0:num_best]), H)
for i in range(len(best_indices)):
    best_indices[i].sort(key = lambda j : -H[i,j])
best_words = [[vocab_rev[i] for i in lst] for lst in best_indices]

In [268]:
artist_df[artist_df.artist==cols[dim1]]

,artist,name
643,d87e52c5-bb8d-4da8-b941-9f4928627dc8,ABBA


In [278]:
artist_df.set_index('artist').loc[np.array(cols[364:])[nmf.components_[dim0,364:]>0],:]

,name
artist,
a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,U2
4d9fb84a-8104-48c1-aa16-21f53c9053cc,Ignite
5f58803e-8c4c-478e-8b51-477f38483ede,Madness
c1e5344e-1bff-4727-9417-a4f55e41b5ff,Loreena McKennitt
96c1edac-1011-4cb8-882c-27248de35071,The Cramps
f46bd570-5768-462e-b84c-c7c993bbf47e,Eagles
c1d4f2ba-cf39-460c-9528-6b827d3417a1,Yes
cf0f4547-ffbd-4011-98ad-0bec9ba022db,Kings of Convenience
f27ec8db-af05-4f36-916e-3d57f91ecf5e,Michael Jackson


In [274]:
np.array(cols[364:])[nmf.components_[dim0,364:]>0]

array(['a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432',
       '4d9fb84a-8104-48c1-aa16-21f53c9053cc',
       '5f58803e-8c4c-478e-8b51-477f38483ede',
       'c1e5344e-1bff-4727-9417-a4f55e41b5ff',
       '96c1edac-1011-4cb8-882c-27248de35071',
       'f46bd570-5768-462e-b84c-c7c993bbf47e',
       'c1d4f2ba-cf39-460c-9528-6b827d3417a1',
       'cf0f4547-ffbd-4011-98ad-0bec9ba022db',
       'f27ec8db-af05-4f36-916e-3d57f91ecf5e',
       'bf0f7e29-dfe1-416c-b5c6-f9ebc19ea810',
       '61604b45-8a91-4e33-a1b6-45d7b1fec4e5',
       'ef6a8aab-9dfe-46ac-a225-67df4601ad69',
       '854a1807-025b-42a8-ba8c-2a39717f1d25',
       'e0ededb4-6085-4f68-90f7-89bc560930a3',
       '3bcff06f-675a-451f-9075-99e8657047e8',
       'b071f9fa-14b0-4217-8e97-eb41da73f598',
       '7746d775-9550-4360-b8d5-c37bd448ce01',
       'd8354b38-e942-4c89-ba93-29323432abc3',
       '9efff43b-3b29-4082-824e-bc82f646f93d',
       '83e59f23-3b0b-4304-834d-5bcafd5df6d2',
       'b88ca659-0393-4a62-abd8-f290e6c7a7e2',
       '16e1f

In [184]:
%%time
print nmf.components_.shape
print nmf.transform(m).shape

(233286L, 100L)
Wall time: 2min 42s
